In [3]:
# | default_exp core

## core


In [26]:
# | hide
from nbdev.showdoc import *
from fasthtml.common import ft_hx, Style

In [48]:
#| export
from pathlib import Path
import httpx
import json

from IPython.display import display, HTML
from fasthtml.common import *

## Get CSS

In [49]:
#| export

def get_css(static_dir=None):
    """Sync OpenProps and OpenProps UI files to local static directory"""
    static_dir = Path(static_dir or 'static')
    css_dir = static_dir / 'css'
    op_dir = css_dir / 'opbeta'
    
    # Create directory structure
    op_dir.mkdir(parents=True, exist_ok=True)
    (op_dir / 'css').mkdir(exist_ok=True)
    
    # Define sources
    OPENPROPS_CDN = "https://unpkg.com/open-props@2.0.0-beta.5"
    OPUI_GITHUB = "https://raw.githubusercontent.com/felix-bohlin/ui/refs/heads/main/src"
    OPUI_GITHUB_API = "https://api.github.com/repos/felix-bohlin/ui/contents/src"
    
    # OpenProps files - still hardcoded as these are specific
    op_files = [
        'index.css',
        'css/media-queries.css',
        'css/sizes/media.css',
        'css/font/lineheight.css',
        'css/color/hues.oklch.css',
        'utilities.css'
    ]
    
    # Create necessary directories for OpenProps files
    for file in op_files:
        (op_dir / Path(file).parent).mkdir(parents=True, exist_ok=True)
    
    # OpenProps UI core files
    core_files = ['normalize.css', 'utils.css', 'theme.css', 'main.css']
    
    # Dynamically discover component directories
    headers = {}
    # Add GitHub token if available (to avoid rate limits)
    # headers = {"Authorization": "token YOUR_GITHUB_TOKEN"}
    
    try:
        response = httpx.get(OPUI_GITHUB_API, headers=headers)
        if response.status_code == 200:
            content_data = response.json()
            component_dirs = []
            for item in content_data:
                if item['type'] == 'dir' and item['name'] not in ['assets', 'lib', 'icons']:
                    component_dirs.append(item['name'])
            print(f"Discovered component directories: {component_dirs}")
        else:
            # Fallback to known component directories
            component_dirs = ['actions', 'data-display', 'feedback', 'inputs', 'text']
            print(f"Using known component directories due to API error: {component_dirs}")
    except Exception as e:
        # Fallback to known component directories
        component_dirs = ['actions', 'data-display', 'feedback', 'inputs', 'text']
        print(f"Exception occurred during directory discovery: {str(e)}")
        print(f"Using known component directories: {component_dirs}")
    
    # Sync OpenProps files
    print("\nSyncing OpenProps files...")
    for file in op_files:
        response = httpx.get(f"{OPENPROPS_CDN}/{file}")
        if response.status_code == 200:
            file_path = op_dir / file
            file_path.parent.mkdir(parents=True, exist_ok=True)
            file_path.write_text(response.text)
            print(f"Synced {file}")
    
    # Sync OpenProps UI core files
    print("\nSyncing OpenProps UI core files...")
    for file in core_files:
        response = httpx.get(f"{OPUI_GITHUB}/{file}")
        if response.status_code == 200:
            (css_dir / file).write_text(response.text)
            print(f"Synced {file}")
    
    # Sync OpenProps UI component files (dynamically)
    print("\nSyncing OpenProps UI component files...")
    for dir_name in component_dirs:
        dir_path = css_dir / dir_name
        dir_path.mkdir(parents=True, exist_ok=True)
        
        # Get the list of files in this component directory
        try:
            response = httpx.get(f"{OPUI_GITHUB_API}/{dir_name}", headers=headers)
            if response.status_code == 200:
                files_data = response.json()
                for file_info in files_data:
                    if file_info['type'] == 'file' and file_info['name'].endswith('.css'):
                        file_name = file_info['name']
                        file_response = httpx.get(f"{OPUI_GITHUB}/{dir_name}/{file_name}")
                        if file_response.status_code == 200:
                            (dir_path / file_name).write_text(file_response.text)
                            print(f"Synced {dir_name}/{file_name}")
                        else:
                            print(f"Failed to download {dir_name}/{file_name}: {file_response.status_code}")
            else:
                print(f"Failed to get file list for {dir_name}: {response.status_code}")
        except Exception as e:
            print(f"Error syncing files from {dir_name} directory: {str(e)}")
    
    print("\nSync complete!")


if __name__ == "__main__":
    get_css()


Discovered component directories: ['actions', 'data-display', 'feedback', 'inputs', 'text']

Syncing OpenProps files...
Synced index.css
Synced css/media-queries.css
Synced css/sizes/media.css
Synced css/font/lineheight.css
Synced css/color/hues.oklch.css
Synced utilities.css

Syncing OpenProps UI core files...
Synced normalize.css
Synced utils.css
Synced theme.css
Synced main.css

Syncing OpenProps UI component files...
Synced actions/button-group.css
Synced actions/button.css
Synced actions/icon-button.css
Synced actions/tab-buttons.css
Synced actions/toggle-button-group.css
Synced data-display/accordion.css
Synced data-display/avatar.css
Synced data-display/badge.css
Synced data-display/card.css
Synced data-display/chip.css
Synced data-display/definition-list.css
Synced data-display/description-list.css
Synced data-display/divider.css
Synced data-display/link.css
Synced data-display/list.css
Synced data-display/table.css
Synced data-display/tooltip.css
Synced feedback/alert.css
Sync

## Update Jupyter Styles

In [50]:

def apply_global_style(css_file):
    """
    Applies a global stylesheet via CDN this will change the behavior of all show() untill notebook is restarted.
    Note: This will affect ALL elements on the page, including previously rendered ones.
    """
    display(HTML(f"<link rel='stylesheet' href='{css_file}'>"))
    return 


apply_global_style("nbs/static/css/main.css")


In [70]:
"""
CSS Flattener - Combines all CSS files from OpenProps UI structure into a single flat file
"""

import os
import re
from pathlib import Path


def flatten_css(css_dir=None, output_file=None):
    """
    Combine all CSS files from OpenProps UI structure into a single flat file
    
    Args:
        css_dir (str): Path to the CSS directory (default: 'static/css')
        output_file (str): Path to the output file (default: 'static/css/flat.css')
    """
    # Set default paths if not provided
    css_dir = Path(css_dir or 'static/css')
    output_file = Path(output_file or css_dir / 'flat.css')
    
    if not css_dir.exists() or not css_dir.is_dir():
        raise ValueError(f"CSS directory not found: {css_dir}")
    
    # Read the main.css file first to determine import order
    main_css_path = css_dir / 'main.css'
    if not main_css_path.exists():
        raise ValueError(f"main.css not found in {css_dir}")
    
    main_css = main_css_path.read_text(encoding='utf-8')
    
    # Extract layer information
    layer_match = re.search(r'@layer\s+(.*?);', main_css)
    layers = []
    if layer_match:
        layers = [layer.strip() for layer in layer_match.group(1).split(',')]
        print(f"Found layers: {', '.join(layers)}")
    
    # Extract import statements
    import_pattern = re.compile(r'@import\s+["\'](.+?)["\'](?:\s+layer\((.*?)\))?;')
    imports = []
    
    for match in import_pattern.finditer(main_css):
        path = match.group(1)
        layer = match.group(2) if match.group(2) else None
        imports.append((path, layer))
    
    # Process imports
    combined_css = []
    processed_files = set()
    
    # Add layer declaration if layers were found
    if layers:
        combined_css.append(f"@layer {', '.join(layers)};")
    
    for path, layer in imports:
        # Convert relative paths
        if path.startswith('./'):
            file_path = css_dir / path[2:]
        elif path.startswith('opbeta/'):
            file_path = css_dir / path
        else:
            file_path = css_dir / path
        
        if not file_path.exists():
            print(f"Warning: File not found: {file_path}")
            continue
        
        # Avoid processing files more than once
        if str(file_path) in processed_files:
            continue
        
        # Read the file
        css_content = file_path.read_text(encoding='utf-8')
        
        # Skip import statements within the imported file
        css_content = re.sub(r'@import\s+["\'].*?["\'].*?;', '', css_content)
        
        # Add layer wrapper if specified
        if layer:
            combined_css.append(f"/* From: {path} */")
            combined_css.append(f"@layer {layer} {{")
            combined_css.append(css_content.strip())
            combined_css.append("}")
        else:
            combined_css.append(f"/* From: {path} */")
            combined_css.append(css_content.strip())
        
        combined_css.append("")  # Empty line for separation
        processed_files.add(str(file_path))
    
    # Save combined CSS to output file
    output_file.parent.mkdir(parents=True, exist_ok=True)
    output_file.write_text("\n".join(combined_css), encoding='utf-8')
    
    print(f"\nFlattened CSS created at: {output_file}")
    print(f"Combined {len(processed_files)} CSS files")


# if __name__ == "__main__":
#     import argparse
    
#     parser = argparse.ArgumentParser(description="Flatten CSS files into a single file")
#     parser.add_argument("--css-dir", "-d", help="Path to CSS directory (default: static/css)")
#     parser.add_argument("--output", "-o", help="Output file path (default: static/css/flat.css)")
    
#     args = parser.parse_args()
    
#     try:
#         flatten_css(args.css_dir, args.output)
#     except Exception as e:
#         print(f"Error: {str(e)}")


In [68]:
css_flat = "https://github.com/Deufel/css/blob/f183a433e7225f466c595e57a7d066c6bbfc25bb/static/css/flat.css"

In [71]:
from IPython.display import HTML

def create_styled_iframe(content, css_url="https://cdn.jsdelivr.net/gh/Deufel/css@030f494da6ab7876da9077f878dfd84fcfbb3bc2/css/main.css", height="300px", width="100%"):
    iframe_html = f"""
    <iframe srcdoc='
        <!DOCTYPE html>
        <html>
        <head>
            <link rel="stylesheet" href="{css_url}">
            <script src="https://unpkg.com/htmx.org@1.9.10"></script>
        </head>
        <body>
            {content}
        </body>
        </html>
    ' style="width: {width}; height: {height}; border: 1px solid #ddd; border-radius: 4px;"></iframe>
    """
    return HTML(iframe_html)

html_content = """
<div class="card elevated">
  <hgroup>
    <h2>Test Component</h2>
  </hgroup>
  <div class="content">
    <p>This is a test of the OpenProps styling</p>
    <button class="button">Primary Button</button>
    <button class="button secondary">Secondary Button</button>
  </div>
</div>
"""
display(create_styled_iframe(html_content))

In [81]:
from IPython.display import HTML

def create_styled_iframe(content, css_url="https://cdn.jsdelivr.net/gh/Deufel/css@ebf95dcad18a553ab53c83f76b5ab026cb91bc21/static/css/main.css", height="300px", width="100%"):
    iframe_html = f"""
    <iframe srcdoc='
        <!DOCTYPE html>
        <html>
        <head>
            <link rel="stylesheet" href="{css_url}">
            <script src="https://unpkg.com/htmx.org@1.9.10"></script>
        </head>
        <body>
            {content}
        </body>
        </html>
    ' style="width: {width}; height: {height}; border: 1px solid #ddd; border-radius: 4px;"></iframe>
    """
    return HTML(iframe_html)

html_content = """
<div class="card elevated padding">
  <hgroup>
    <h2>Test Component</h2>
  </hgroup>
  <div class="content">
    <p>This is a test of the OpenProps styling</p>
    <button class="button">Default</button>
    <button class="button outlined">Outlined</button>
    <button class="button outlined" disabled>Outlined & disabled</button>
    <button class="button tonal">Outlined</button>
    <button class="button elevated">Outlined</button>
  </div>
</div>
"""
display(create_styled_iframe(html_content))

In [87]:
from fasthtml.common import *

def flat_css_link():
    return Link(
        rel="stylesheet",
        href="https://cdn.jsdelivr.net/gh/Deufel/css@f183a433e7225f466c595e57a7d066c6bbfc25bb/static/css/flat.css",
        type="text/css"
    )

def card_examples():
    return Div(
        H2("Card Examples"),
        Div(
            Button("btn", cls="button outlined margin padding"),
            Div(Div("Outlined", cls="content"), cls="card outlined"),
            Div(Div("Tonal", cls="content"), cls="card tonal"),
            Div(Div("Elevated", cls="content"), cls="card elevated"),
        ),
        P("These cards demonstrate different variants from the Flat CSS library.")
    )

# Preview with Link approach
show(
    flat_css_link(),
    card_examples()
)


In [ ]:
            <link rel="stylesheet" href="{css_url}">
